# Leaf Disease Detection using CNN

## Setup

### Import Tensorflow and Keras

we will be using Keras for this project 

In [ ]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

### Declare constants

We will use these constants to quickly tune our model and other parameters

In [ ]:
DATASET_DIR= "./data/tomato"
EPOCHS = 500
BATCH_SIZE = 32
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3
IMAGE_SHAPE=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
NUM_CLASSES=10
AUTOTUNE = tf.data.AUTOTUNE

## Prepare Dataset

### Import images and split for training

Import images from local dataset using `image_dataset_from_directory` from `keras.utils`.<br>
Next step is to split the dataset into training and validation set.

Note: The Dataset used here is setup in following hierarchy in accordance to `image_dataset_from_directory` API
```
data/
├─ tomato/
│  ├─ train/
│  │  ├─ Disease Name/
│  │  │  ├─ 1.jpg
│  ├─ test/
│  │  ├─ Disease Name/
│  │  │  ├─ 2.jpg

```

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
  F"{DATASET_DIR}/train",
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=BATCH_SIZE,
)

val_ds = keras.utils.image_dataset_from_directory(
  F"{DATASET_DIR}/train",
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=BATCH_SIZE,
)

### Preview Data

We can ever preview some images with their labels. `image_dataset_from_directory` automatically infers labels from directory structure so we will print out the labels to confirm.

In [ ]:
class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Image Preprocessing

Here we setup a `keras.Sequential` pipeline to preprocess the incoming data.<br>

First we Resize the images to desired shape.<br>
Then we Rescale the image to map the RGB values ranging from `[0, 255]` to `[0, 1]`


In [ ]:
preprocessing = keras.Sequential([
    keras.layers.Resizing(IMAGE_HEIGHT, IMAGE_WIDTH),
    keras.layers.Rescaling(1./255),
])

### Data Augmentation

Data augmentation drastically increases accuracy, reduces overfitting and let's you train the model with scarce dataset.<br>

Here we setup a sequential pipeline to apply augmentation to our images.

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomTranslation(0.2, 0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomBrightness(0.4),
    keras.layers.RandomContrast(0.4),
])

train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

## Build the CNN

### Create CNN layers

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=(None, None, 3)),

    preprocessing,

    keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    # keras.layers.BatchNormalization(momentum=0.87),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    # keras.layers.BatchNormalization(momentum=0.87),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    # keras.layers.BatchNormalization(momentum=0.87),
    keras.layers.MaxPooling2D(),

    keras.layers.GlobalAveragePooling2D(),

    keras.layers.Dropout(0.15),

    keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.summary(
    expand_nested=True,
)

### Training The Model

#### Prefetch Data

Cache and prefetch data to reduce load and improve learning time

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#### Training

In [ ]:
earlystop_loss = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[earlystop_loss],
)

## Results

### Graph The Learning Curve

In [ ]:
from matplotlib import pyplot as plt

print("Epochs run:", len(history.history["loss"]))

acc = history.history["sparse_categorical_accuracy"]
val_acc = history.history["val_sparse_categorical_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.ylim((0, 1))
plt.plot(epochs, acc, label="Training accurarcy")
plt.plot(epochs, val_acc, label="Validation accurarcy")
plt.title("Training and Validation accurarcy")
plt.legend()

# Train and validation loss
plt.subplot(1, 2, 2)

plt.plot(epochs, loss, label="Training loss")
plt.plot(epochs, val_loss, label="Validation loss")
plt.title("Training and Validation loss")
plt.legend()
plt.show()

### Evaluate Model Accuracy

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(F"{DATASET_DIR}/test")

results = model.evaluate(test_ds, verbose=0)

print("Test accuracy:", results[1]*100)
print("Test loss:", results[0]*100)

### (Optional) Plot Model Architecture

In [ ]:
dot_img_file = '/tmp/model_architecture.png'
tf.keras.utils.plot_model(
    model,
    to_file=dot_img_file,
    show_shapes=True,
    show_layer_names=False,
    expand_nested=True
)